<a href="https://colab.research.google.com/github/jaredmullane/LLM_Class/blob/main/Jared_TECH16_LLM_Lecture2_prepared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [3]:
from openai import OpenAI
from google.colab import userdata

open_ai_key = userdata.get('OpenAI')
client = OpenAI(api_key=open_ai_key)


# Prompt engineering - few shot

In [4]:
def chat(message):
    """
    Send a message to the OpenAI GPT-3.5 model and return its response.

    This function interacts with the OpenAI API, specifically using the GPT-3.5-turbo model. It takes a user's message as input, sends it to the model, and returns the model's text-only response. The function ensures the AI's output is concise by providing a system-level instruction.

    Parameters:
    message (str): A string containing the user's message to the AI.

    Returns:
    str: The text response generated by the GPT-3.5 model.
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=0.3,
        # response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": """Below are examples of text messages and their classifications. After studying these examples, please classify the new text message at the end.

              Example 1:

              Text: "Can you send me the files by tomorrow? It's not urgent, but I'd like to review them."
              Classification: Non-Urgent
              Example 2:

              Text: "Please review the final report ASAP! We need it ready by our meeting in the morning!"
              Classification: Urgent
              Example 3:

              Text: "Let's schedule a meeting for next week to discuss the project. No rush."
              Classification: Non-Urgent
              Example 4:

              Text: "URGENT: The server is down and needs immediate attention!"
              Classification: Urgent
              """},
            {"role": "user", "content": f"Classify the following message: {message}"}
        ]
    )

    text_only = response.choices[0].message.content
    return text_only


In [ ]:
chat("Reminder: Tomorrow's team meeting has been postponed. Please update your calendars")

'Classification: Non-Urgent'

### JSON mode and Log Probs

In [14]:
def chat(message):
    """
    Send a message to the OpenAI GPT-3.5 model and return its response.

    This function interacts with the OpenAI API, specifically using the GPT-3.5-turbo model. It takes a user's message as input, sends it to the model, and returns the model's text-only response. The function ensures the AI's output is concise by providing a system-level instruction.

    Parameters:
    message (str): A string containing the user's message to the AI.

    Returns:
    str: The text response generated by the GPT-3.5 model.
    """

    response = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        response_format={ "type": "json_object" },
        logprobs=True,
        messages=[
            {"role": "system", "content": """
            You are an assistant that can determine if text is in English or Spanish.

            Below are examples of text messages and their classifications. After studying these examples, please classify the new text message at the end.

              Example 1:

              Text: "Hello, I am from New York City."
              Classification: English

              Example 2:

              Text: "Soy de la ciudad de nueva york"
              Classification: Spanish

              Example 3:

              Text: "I enjoy eating hamburgers"
              Classification: English

              Example 4:

              Text: "disfruto comiendo hamburguesas"
              Classification: Spanish

              """},
            {"role": "user", "content": f"Classify the following message as English or Spanish and return with probability as JSON: {message}"}
        ]
    )
    text_only = response.choices[0].message.content
    return text_only


In [15]:
response = chat("My favorite building in New York is the Empire State")

In [16]:
print(response)

{
  "Text": "My favorite building in New York is the Empire State",
  "Classification": "English",
  "Probability": 0.99
}


In [17]:
response = chat("Mi edificio favorito en Nueva York es el Empire State.")

In [18]:
print(response)

{
  "classification": "Spanish",
  "probability": 0.99
}


In [21]:
response = chat("Me gusta eating burritos, tacos, and queso")

In [22]:
print(response)

{
  "text": "Me gusta eating burritos, tacos, and queso",
  "classification": "Spanish",
  "probability": 0.6
}


# Chain of thought reasoning

In [9]:
def chat(message):
    """
    Send a message to the OpenAI GPT model and return its response.

    This function interacts with the OpenAI API, specifically using the GPT model. It takes a user's message as input, sends it to the model, and returns the model's text-only response. The function ensures the AI's output is concise by providing a system-level instruction.

    Parameters:
    message (str): A string containing the user's message to the AI.

    Returns:
    str: The text response generated by the GPT model.
    """

    response = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[
            {"role": "system", "content": """
            You are a helpful assistant classifations.

            Below are examples of questions and how to calculate the answer

              Example 1: Arithmetic Problem
              Prompt: "If a shirt costs $20 and there is a 10% discount, how much does the shirt cost after the discount?"
              Chain of Thought Answer:
                Calculate the amount of discount: 10% of $20 is $2.
                Subtract the discount from the original price: $20 - $2 = $18.
                The shirt costs $18 after the discount.

              Example 2: Logic Puzzle
              Prompt: "There are four apples and you take away three. How many apples do you have?"
              Chain of Thought Answer:
                You start with four apples.
                You take away three apples.
                After taking three, you now have those three apples.
                You have 3 apples

              """},
            {"role": "user", "content": f"Answer the following question: {message}"}
        ]
    )
    text_only = response.choices[0].message.content
    return text_only


In [10]:
print(chat("If a tree absorbs 48 pounds of CO2 a year, how much CO2 will 10 trees absorb in a year?"))

To calculate how much CO2 10 trees will absorb in a year:

1. Understand first that one tree absorbs 48 pounds of CO2 a year.
2. To find out the total CO2 absorption by 10 trees, multiply the amount absorbed by one tree by 10.

Calculation:
48 pounds/tree/year * 10 trees = 480 pounds of CO2/year

Therefore, 10 trees will absorb 480 pounds of CO2 in a year.
